Descripción del proyecto

Telecomunicaciones Interconnect busca pronosticar su tasa de cancelación de clientes. 
Si se pronostica que un usuario va a cancelar se le ofrecerán códigos promocionales y planes especiales. 

In [2]:
#Carga de librerías
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

from  sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgb


In [3]:
#Carga de datos
#Datasets: contract, internet, personal, phone

#Directorio de los archivos 
directorio = "C:/Users/david/1.Archivos-Directorios/Python/datasets/s18/final_provider"
contract = pd.read_csv(directorio + "/contract.csv")
internet = pd.read_csv(directorio + "/internet.csv")
personal = pd.read_csv(directorio + "/personal.csv")
phone = pd.read_csv(directorio + "/phone.csv")

In [4]:
#Exploración 
display(contract.head())
print(contract.info())

display(internet.head())
print(internet.info())

display(personal.head())
print(personal.info())

display(phone.head())
print(phone.info())

print("No hay nulos en los datasets, los números de registros varían entre ellos.")

#Duplicados
print(f"\nDuplicados en contract: {contract.duplicated().sum()}")
print(f"Duplicados en internet: {internet.duplicated().sum()}")
print(f"Duplicados en personal: {personal.duplicated().sum()}")
print(f"Duplicados en phone: {phone.duplicated().sum()}")
print("Sin duplicados en los datasets.")

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
None


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
None


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None
No hay nulos en los datasets, los números de registros varían entre ellos.

Duplicados en contract: 0
Duplicados en internet: 0
Duplicados en personal: 0
Duplicados en phone: 0
Sin duplicados en los datasets.


In [5]:
#Descripción estadística de los datasets
print("Contract: \n", contract.describe())
print("internet: \n", internet.describe())
print("Personal: \n",personal.describe())
print("Phone: \n",phone.describe())
#Procesar a numéricos los booleanos 

Contract: 
        MonthlyCharges
count     7043.000000
mean        64.761692
std         30.090047
min         18.250000
25%         35.500000
50%         70.350000
75%         89.850000
max        118.750000
internet: 
         customerID InternetService OnlineSecurity OnlineBackup  \
count         5517            5517           5517         5517   
unique        5517               2              2            2   
top     3186-AJIEK     Fiber optic             No           No   
freq             1            3096           3498         3088   

       DeviceProtection TechSupport StreamingTV StreamingMovies  
count              5517        5517        5517            5517  
unique                2           2           2               2  
top                  No          No          No              No  
freq               3095        3473        2810            2785  
Personal: 
        SeniorCitizen
count    7043.000000
mean        0.162147
std         0.368612
min         0.000000


Cambios a hacer en los datasets. 

1. Tipo de datos. El dataset contract: tiene fechas bajo el tipo de dato objeto, cargo total debe ser numérico y es objeto. 

2. Múltiples variables booleanas manejadas en distintos formatos: como números ejemplo SeniorCitizen 0 o 1; o como objetos ejemplo MultipleLines No o Yes. (Es útil homogenizar) 

3. Las variables podrían seguir formato snakecase (opcional)

In [6]:
#Función para procesar variables si y no
def yn_vars(x):    
    return 1 if str(x).strip().lower() == "yes" else 0

In [7]:
#Procesar los datos #Se puede hacer un pipeline para esto, pero por ahora lo haré manualmente.

#Contract
contract_encoded = contract.copy()
contract_encoded["BeginDate"] = pd.to_datetime(contract_encoded["BeginDate"])
contract_encoded["EndDate"] = pd.to_datetime(contract_encoded["EndDate"], errors='coerce')
#Revisar rango de fechas. 
print(contract_encoded["EndDate"].dt.year.value_counts())
print(contract_encoded["BeginDate"].dt.year.value_counts())
#Crear variable de duración del contrato
max_date = contract_encoded["EndDate"].dropna().max()
contract_encoded["EffectiveEnd"] = contract_encoded["EndDate"].fillna(max_date)

contract_encoded["TenureDays"] = (contract_encoded["EffectiveEnd"] - contract_encoded["BeginDate"]).dt.days
contract_encoded["TenureMonths"] = contract_encoded["TenureDays"] // 30
#Valores útiles para el modelo 
contract_encoded["BeginMonth"] = contract_encoded["BeginDate"].dt.month
contract_encoded["BeginYear"] = contract_encoded["BeginDate"].dt.year
contract_encoded["EndMonth"] = contract_encoded["EndDate"].dt.month
contract_encoded["EndYear"] = contract_encoded["EndDate"].dt.year
contract_encoded["EndMonth"] = contract_encoded["EndMonth"].fillna(0).astype(int)
contract_encoded["EndYear"] = contract_encoded["EndYear"].fillna(0).astype(int)
contract_encoded["IsActive"] = contract_encoded["EndDate"].isna().astype(int)  # 1=activo, 0=terminó #Se eliminó esta columna porque generó data leakage al predecir el churn.
contract_encoded["churn"] = 1 - contract_encoded["IsActive"] #Variable objetivo, 1=churn, 0=no churn. 

contract_encoded["contract_type"] = contract_encoded["Type"].apply(lambda x:2 if str(x).strip().lower() == "two year" else 1 if str(x).strip().lower() == "one year" else 0)

#Payment method, total charge to float
contract_encoded["paper"] = contract_encoded["PaperlessBilling"].apply(yn_vars)
contract_encoded["Total"] = pd.to_numeric(contract_encoded["TotalCharges"], errors='coerce')

#Convertir las fechas a datetime y tiempos de duración como cliente.

#Remover las columnas a no emplear "EndDate", "Type"
contract_encoded.drop(["EndMonth", "TenureDays","IsActive", "EffectiveEnd", "BeginDate", "EndDate", "Type", "PaymentMethod", "TotalCharges", "PaperlessBilling", "EndYear"], axis=1, inplace=True) #Nota: puedo no ocupar PaymentMethod y alcanzar las métricas? 
display(contract_encoded.head())


#internet
internet_encoded = internet.copy()
internet_encoded["InternetService"] = internet_encoded["InternetService"].apply(lambda x: 2 if str(x).strip().lower() == "fiber optic" else 1 if str(x).strip().lower() == "dsl" else 0)
internet_encoded["OnlineSecurity"] = internet_encoded["OnlineSecurity"].apply(yn_vars)
internet_encoded["OnlineBackup"] = internet_encoded["OnlineBackup"].apply(yn_vars)
internet_encoded["DeviceProtection"] = internet_encoded["DeviceProtection"].apply(yn_vars)
internet_encoded["TechSupport"] = internet_encoded["TechSupport"].apply(yn_vars)
internet_encoded["StreamingTV"] = internet_encoded["StreamingTV"].apply(yn_vars)
internet_encoded["StreamingMovies"] = internet_encoded["StreamingMovies"].apply(yn_vars)

#Remover las columnas a no emplear "CustomerID", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"
internet_encoded.drop(["InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"], axis=1, inplace=True)
display(internet_encoded.head())

#personal
personal_encoded = personal.copy()
personal_encoded["gender"] = personal_encoded["gender"].apply(lambda x: 1 if str(x).strip().lower() == "male" else 0)
personal_encoded["Partner"] = personal_encoded["Partner"].apply(yn_vars)
personal_encoded["Dependents"] = personal_encoded["Dependents"].apply(yn_vars)
#Remover las columnas a no emplear "Partner", "Dependents".

#phone
phone_encoded = phone.copy()
phone_encoded["multiplelines"] = phone_encoded["MultipleLines"].apply(yn_vars)
phone_encoded.drop("MultipleLines", axis=1, inplace=True)  

#Unir los valores
df_merged = contract_encoded \
    .merge(internet_encoded, on="customerID", how="left") \
    .merge(personal_encoded, on="customerID", how="left") \
    .merge(phone_encoded, on="customerID", how="left")

EndDate
2019.0    1409
2020.0     460
Name: count, dtype: int64
BeginDate
2019    1957
2014    1344
2018    1030
2015     852
2017     845
2016     763
2020     244
2013       8
Name: count, dtype: int64


C:\Users\david\AppData\Local\Temp\ipykernel_9904\1372951510.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  contract_encoded["EndDate"] = pd.to_datetime(contract_encoded["EndDate"], errors='coerce')


,customerID,MonthlyCharges,TenureMonths,BeginMonth,BeginYear,churn,contract_type,paper,Total
0,7590-VHVEG,29.85,0,1,2020,0,0,1,29.85
1,5575-GNVDE,56.95,33,4,2017,0,1,0,1889.50
2,3668-QPYBK,53.85,2,10,2019,1,0,1,108.15
3,7795-CFOCW,42.30,44,5,2016,0,1,0,1840.75
4,9237-HQITU,70.70,2,9,2019,1,0,1,151.65


,customerID
0,7590-VHVEG
1,5575-GNVDE
2,3668-QPYBK
3,7795-CFOCW
4,9237-HQITU


In [8]:
display(df_merged.head())
df_merged.info() #Tengo pocos valores faltantes en multiplelines y total. 

#Remover los 10 registros con valores faltantes en Total
df_merged = df_merged.dropna(subset=["Total"])

#Imputar valores faltantes por cercanía KNN
imputer = KNNImputer(n_neighbors=5)
ml_imputado = imputer.fit_transform(df_merged[["multiplelines"]])
df_merged["multiplelines"] = ml_imputado.round().astype(int)

df_merged.info() #Verifico que no hay valores faltantes

#Remover customerID porque no es relevante para el modelo predictivo
df_merged.drop("customerID", axis=1, inplace=True)
display(df_merged.head())

#Proporción de clases
print("Proporción de churn (1) y no churn (0):")
print(df_merged["churn"].value_counts(normalize=True))
print("Aproximadamente 3 veces más clientes activos que desertados.")

,customerID,MonthlyCharges,TenureMonths,BeginMonth,BeginYear,churn,contract_type,paper,Total,gender,SeniorCitizen,Partner,Dependents,multiplelines
0,7590-VHVEG,29.85,0,1,2020,0,0,1,29.85,0,0,1,0,NaN
1,5575-GNVDE,56.95,33,4,2017,0,1,0,1889.50,1,0,0,0,0.0
2,3668-QPYBK,53.85,2,10,2019,1,0,1,108.15,1,0,0,0,0.0
3,7795-CFOCW,42.30,44,5,2016,0,1,0,1840.75,1,0,0,0,NaN
4,9237-HQITU,70.70,2,9,2019,1,0,1,151.65,0,0,0,0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customerID      7043 non-null   object 
 1   MonthlyCharges  7043 non-null   float64
 2   TenureMonths    7043 non-null   int64  
 3   BeginMonth      7043 non-null   int32  
 4   BeginYear       7043 non-null   int32  
 5   churn           7043 non-null   int64  
 6   contract_type   7043 non-null   int64  
 7   paper           7043 non-null   int64  
 8   Total           7032 non-null   float64
 9   gender          7043 non-null   int64  
 10  SeniorCitizen   7043 non-null   int64  
 11  Partner         7043 non-null   int64  
 12  Dependents      7043 non-null   int64  
 13  multiplelines   6361 non-null   float64
dtypes: float64(3), int32(2), int64(8), object(1)
memory usage: 715.4+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 14 columns

,MonthlyCharges,TenureMonths,BeginMonth,BeginYear,churn,contract_type,paper,Total,gender,SeniorCitizen,Partner,Dependents,multiplelines
0,29.85,0,1,2020,0,0,1,29.85,0,0,1,0,0
1,56.95,33,4,2017,0,1,0,1889.50,1,0,0,0,0
2,53.85,2,10,2019,1,0,1,108.15,1,0,0,0,0
3,42.30,44,5,2016,0,1,0,1840.75,1,0,0,0,0
4,70.70,2,9,2019,1,0,1,151.65,0,0,0,0,0


Proporción de churn (1) y no churn (0):
churn
0    0.734215
1    0.265785
Name: proportion, dtype: float64
Aproximadamente 3 veces más clientes activos que desertados.


In [9]:
#Dividir los datos en entrenamiento y prueba
X = df_merged.drop("churn", axis=1)
y = df_merged["churn"]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #0.8 entrenamiento y validación, 0.2 prueba. 
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val) #75% entrenamiento, 25% validación, 20% prueba
#Equivalente a 60% entrenamiento, 20% validación, 20% prueba. 

#Verificar tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]}")
print(f"Tamaño del conjunto de validación: {X_val.shape[0]}")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]}")

#Verificar proporción de clases en entrenamiento y validación
print("Proporción de churn en entrenamiento:")  
print(y_train.value_counts(normalize=True))
print("Proporción de churn en validación:")
print(y_val.value_counts(normalize=True))   
print("Proporción de churn en prueba:")
print(y_test.value_counts(normalize=True))
print("Las proporciones de churn son similares en entrenamiento, validación y prueba.")


Tamaño del conjunto de entrenamiento: 4218
Tamaño del conjunto de validación: 1407
Tamaño del conjunto de prueba: 1407
Proporción de churn en entrenamiento:
churn
0    0.734234
1    0.265766
Name: proportion, dtype: float64
Proporción de churn en validación:
churn
0    0.734186
1    0.265814
Name: proportion, dtype: float64
Proporción de churn en prueba:
churn
0    0.734186
1    0.265814
Name: proportion, dtype: float64
Las proporciones de churn son similares en entrenamiento, validación y prueba.


In [10]:
#Modelado 

#Regresión logística
model_lr = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
model_lr.fit(X_train, y_train)
#Predicciones y evaluación
y_pred = model_lr.predict(X_val)
#Métricas de evaluación
print("Métricas de evaluación para Regresión Logística:")
print(classification_report(y_val, y_pred))

#ROC y AUC
y_scores = model_lr.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_scores)
print(f"AUC para Regresión Logística en validación: {roc_auc:.4f}\n")

Métricas de evaluación para Regresión Logística:
              precision    recall  f1-score   support

           0       0.90      0.69      0.78      1033
           1       0.48      0.79      0.59       374

    accuracy                           0.71      1407
   macro avg       0.69      0.74      0.69      1407
weighted avg       0.79      0.71      0.73      1407

AUC para Regresión Logística en validación: 0.8174



In [11]:
#Random Forest
modelo_rf = RandomForestClassifier(random_state=42, class_weight='balanced')

#Grid Search
param_grid = {  
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(modelo_rf, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
#Modelo con los mejores parámetros
best_rf_model = grid_search.best_estimator_

#Predicciones y evaluación
y_pred_rf = best_rf_model.predict(X_val)
#Métricas de evaluación
print("Métricas de evaluación para Random Forest:")
print(classification_report(y_val, y_pred_rf))

Mejores parámetros encontrados:
{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Métricas de evaluación para Random Forest:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      1033
           1       0.79      0.60      0.69       374

    accuracy                           0.85      1407
   macro avg       0.83      0.77      0.79      1407
weighted avg       0.85      0.85      0.85      1407



In [12]:
#ROC y AUC - rf
y_scores_rf = best_rf_model.predict_proba(X_val)[:, 1]
roc_auc_rf_val = roc_auc_score(y_val, y_scores_rf)
print(f"AUC para Random Forest en validación: {roc_auc_rf_val:.4f}\n")

#Prueba con test
#Predicción y evaluación en el conjunto de prueba
y_pred_rf_test = best_rf_model.predict(X_test)
#Métricas de evaluación
print("Métricas de evaluación para Random Forest en prueba:")
print(classification_report(y_test, y_pred_rf_test))
#ROC y AUC en el conjunto de prueba
y_scores_rf_test = best_rf_model.predict_proba(X_test)[:, 1]
print(f"AUC para Random Forest en prueba: {roc_auc_score(y_test, y_scores_rf_test):.4f}")

print("El modelo Random Forest se desempeña mejor que la regresión logística y se mantiene estable en el conjunto de prueba con una reducción de un decimal desde .89 a .88.")


AUC para Random Forest en validación: 0.8947

Métricas de evaluación para Random Forest en prueba:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1033
           1       0.83      0.60      0.70       374

    accuracy                           0.86      1407
   macro avg       0.85      0.78      0.80      1407
weighted avg       0.86      0.86      0.85      1407

AUC para Random Forest en prueba: 0.8869
El modelo Random Forest se desempeña mejor que la regresión logística y se mantiene estable en el conjunto de prueba con una reducción de un decimal desde .89 a .88.


In [13]:
#lgbm
model_lgbm = lgb.LGBMClassifier(random_state=42, class_weight="balanced", verbosity =-1)

#Param grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'min_child_samples': [10, 20],  # correcto
    # no uses min_samples_leaf ni min_samples_split
}

#Grid Search
grid_search_2 = GridSearchCV(model_lgbm, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose =0)

#Entrenamiento silencioso
# Silenciar stdout + stderr durante el fit (importante para LightGBM)
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull), redirect_stderr(fnull):
        grid_search_2.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_search_2.best_params_)
#Modelo con los mejores parámetros
best_lgbm_model = grid_search_2.best_estimator_

#Predicciones y evaluación
y_pred_lgbm = best_lgbm_model.predict(X_val)
#Métricas de evaluación
print("Métricas de evaluación para lgbm en validación:")
print(classification_report(y_val, y_pred_lgbm))

NameError: name 'os' is not defined

In [14]:
#lgbm
model_lgbm = lgb.LGBMClassifier(random_state=42, is_unbalanced=True)

#Grid Search
grid_search_2 = GridSearchCV(model_lgbm, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search_2.fit(X_train, y_train)
print("Mejores parámetros encontrados:")
print(grid_search_2.best_params_)
#Modelo con los mejores parámetros
best_lgbm_model = grid_search_2.best_estimator_

#Predicciones y evaluación
y_pred_lgbm = best_lgbm_model.predict(X_val)
#Métricas de evaluación
print("Métricas de evaluación para Random Forest:")
print(classification_report(y_val, y_pred_rf))

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=2. Current value: min_data_in_leaf=2
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=2. Current value: min_data_in_leaf=2
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Info] Number of positive: 1121, number of negative: 3097
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000720 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 620
[LightGBM] [Info] Number of data points in the train set: 4218, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265766 -> 

In [ ]:
#ROC y AUC -lgbm
y_scores_lgbm = best_lgbm_model.predict_proba(X_val)[:, 1]
roc_auc_lgbm_val = roc_auc_score(y_val, y_scores_lgbm)
print(f"AUC para Random Forest en validación: {roc_auc_lgbm_val:.4f}\n")

#Prueba con test
#Predicción y evaluación en el conjunto de prueba
y_pred_lgbm_test = best_lgbm_model.predict(X_test)
#Métricas de evaluación
print("Métricas de evaluación para Random Forest en prueba:")
print(classification_report(y_test, y_pred_lgbm_test))
#ROC y AUC en el conjunto de prueba
y_scores_lgbm_test = best_lgbm_model.predict_proba(X_test)[:, 1]
print(f"AUC para Random Forest en prueba: {roc_auc_score(y_test, y_scores_lgbm_test):.4f}")

print("El modelo LGBm se desempeña mejor que la regresión logística y random forest y se mantiene estable entre conjunto de validación y prueba con un roc-auc de 0.93")

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=2. Current value: min_data_in_leaf=2
[LightGBM] [Warning] Unknown parameter: min_samples_split
AUC para Random Forest en validación: 0.9393

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=2. Current value: min_data_in_leaf=2
[LightGBM] [Warning] Unknown parameter: min_samples_split
Métricas de evaluación para Random Forest en prueba:
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      1033
           1       0.90      0.71      0.79       374

    accuracy                           0.90      1407
   macro avg       0.90      0.84      0.86      1407
weighted avg       0.90      0.90      0.90      1407

[LightGBM] [Warning] Unknown parameter: is_unbalanced


In [ ]:
#Mejor modelo predictor de deserción 
print(best_lgbm_model)

#Características de la predicción
print("Características de la predicción:")
print("71% de sensibilidad a deserción, 93% de discernimiento entre clases.")

LGBMClassifier(is_unbalanced=True, max_depth=10, min_samples_leaf=2,
               min_samples_split=2, n_estimators=200, random_state=42)


Conclusiones

Todos los modelos se desempeñan eficientemente en predecir si un cliente va a desertar o no.

El más sensible de los modelos tiene un discernimiento del 93% en datos nuevos. 

Es interesante destacar que la sensibilidad o capacidad de detectar los clientes desertores en los modelos se queda entre el 60% y el 71% en el mejor modelo esta métrica puede ser considerada para aumentar la sensbilidad y rescatar aún más clientes de ser requerido. 